In [ ]:
import re
import json
from bs4 import BeautifulSoup
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from requests.sessions import Session
from time import sleep

In [ ]:
def download(url, params={}, headers={}, method='GET', maximum=2):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [ ]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

In [ ]:
# 큐로 push, pop 구현할 거
def new_download(url):
    _ = urlparse(url)
    print(can_fetch(_[0]+'://'+_[1], _[2])[0])
    
    return download(url, headers=headers)

In [ ]:
session = Session()
session.cookies.clear()

for i, _ in enumerate(naver):
    session.cookies.set(naver[i]['name'], naver[i]['value'],
                       domain=naver[i]['domain'], path=naver[i]['path'])

In [ ]:
# 메일 화면이 뜨면서 로그인 된 것을 확인할 수 있음
resp = session.get('https://mail.naver.com')
resp.text

In [ ]:
# 어제까지 한 내용 : 쿠키의 중요성
# 쿠키로 로그인을 어떻게 하는가! 왜 이게 중요한 것인가
# 오전에 할 내용 ) 스팸 메일

In [ ]:
# 요새 웹들은 리엑트와 같은 프레임워크를 사용하므로
# 실제 메일 데이터는 뒤에서 따로 가져오게 됨.

In [ ]:
url = 'https://mail.naver.com/json/list/'

params = {
    'page':1,
    'sortField':1,
    'sortType':0,
    'folderSN':5,
    'type':None,
    'isUnread':'false',
    'u':'dimoteo333'
}

In [ ]:
resp = request('get', url=url, params=params)

In [ ]:
# 세션(쿠키값)없이 보내면 로그인이 필요하다고 나옴
resp.text

In [ ]:
resp = session.request('post', url=url, params=params)

In [ ]:
resp.json().keys()

In [ ]:
resp.json()['mailData'][0].keys()

In [ ]:
for _ in resp.json()['mailData']:
    print(_['subject'])

In [ ]:
# 의문점 : POST 방식으로 데이터를 보냈는데 주소 방식은 GET 방식임.
# 그렇지만 GET 방식으로 request를 보내면 서버가 응답을 끊음
# 소켓을 쓰는 경우가 아니라면 왠만해서는 다 가져올 수 있음.

In [ ]:
from selenium.webdriver import Chrome, ChromeOptions # PhantomJS or ChormeOptions의 headless

In [ ]:
# 코드에 의해 돌아가는 별개의 브라우져임.
# 원래 목적은 브라우져 테스트 용인데, 어쩌다 보니 수집용으로 쓰이고 있는거
# 원래는 requests 패키지를 사용해 수집하는 것이 맞는 것
driver = Chrome()

In [ ]:
driver.get('https://www.naver.com') # 네이버로 이동함

In [ ]:
# 이미 렌더링된 오브젝트 모델 (DOM)을 가지고 있음.
for _ in driver.find_elements_by_css_selector('button'): # 로그인 버튼을 찾아보자
    print(_.text)
    
# 없음.. 버튼이 아닌 듯?

In [ ]:
driver.find_element_by_css_selector('a.link_login').click()
# 로그인 화면으로 넘어감!

In [ ]:
# HTML 코드 레벨에서 찾는 방법 - 이게 훨씬 빠름.
dom = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
for _ in  dom.find_all(text=re.compile(r'로그인')):
    print(_.find_parent())
    
# 맨 마지막 값 : 로그인 버튼 바로가기

In [ ]:
# 정확히는 form 태그를 꼭 먼저 찾아야함! 거기에 input parameter가 어떻게 들어가는지 확인해야 하므로
#
# 이를 활용하면 예매 매크로를 짤 수 있음.
for _ in driver.find_elements_by_css_selector('form > input[name]'):
    print(_.tag_name, _.get_attribute('name'))

In [ ]:
driver.find_element_by_name('id').clear()
driver.find_element_by_name('id').send_keys('dimoteo333')

In [ ]:
for _ in driver.find_elements_by_css_selector('form a, form button'):
    print(_.tag_name, _.text, _.get_attribute('id'))

In [ ]:
form
 input -> param
 submit
 button
 a

이런 구조...

In [ ]:
driver.find_element_by_name('pw').clear()
driver.find_element_by_name('pw').send_keys('abcde')

In [ ]:
driver.find_element_by_css_selector(
    'form input[type=submit]').get_attribute('value')
driver.find_element_by_css_selector(
    'form input[type=submit]').click()

In [ ]:
# 백날 로그인 상태 유지 눌러도 안된다
# 백날 자주 사용하는 기기 등록해도 소용 없음.
# 실제 로그인을 시도해도, 마우스의 움직임이 없기 때문에 자동방지 입력문자를 입력해야만 함.
# 이를 해결하기 위해서는 계정의 보안 등급을 굉장히 낮춰야 로그인됨
# 예) 구글의 reCaptcha
#
# 기본적으로 자동입력 방지 문자로 글자를 살짝 기울인 정도는 뚫는 패키지가 존재

In [ ]:
# 지금 보고 있는 창의 ID임. 새 탭을 띄우든, 드라이버를 다시 키든 매번 바뀔 예정
# => 등록해도 소용이 없음.
driver.current_window_handle

In [ ]:
# 쿠키를 더 간단하게 이용하기
naver = driver.get_cookies()

In [ ]:
session.cookies.get_dict()

In [ ]:
session.cookies.clear()
session.cookies.get_dict()

In [ ]:
for _ in naver:
    print(_['value'])

In [ ]:
driver = Chrome()

In [ ]:
for _ in naver:
    session.cookies.set(_['name'], _['value'],
                        domain=_['domain'], path=_['path'])

In [ ]:
resp = session.request('post', url=url, params=params)
resp.json()

In [ ]:
# 즉, 이 방법을 이용한다면 쿠키만 가지고 네이버 로그인한 채로 이용할 수 있다.
driver.delete_all_cookies()

In [ ]:
driver.get('https://www.naver.com')

In [ ]:
for _ in naver:
    driver.add_cookie(_)

In [ ]:
driver.get('https://www.naver.com')

In [ ]:
driver.get('https://mail.naver.com')

In [ ]:
dom = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
dom.find(text=re.compile('새로운 기기에서')).find_parent().find_parent()\
.find_parent().find_parent().find_parent() # ?

In [ ]:
len(dom.select('.mailList > li')) # 위에서 찾은게 <ol class=mailList에 있는 자손중에 li를 찾으면 몇개 있나 알 수 있음.

In [ ]:
for _ in dom.select('.mailList > li li > label'):
    print(_.text)

In [ ]:
# BeautifulSoup이 Driver보다 더 빠름. 
# 물론 지금 어짜피 렌더링 되어있는 화면이 떠있기 때문에 굳이 beautifulsoup 을 쓸 이유는 없다.

In [ ]:
driver.find_element_by_xpath('/html') # 최상단의 자식 노드는 html일 것.

In [ ]:
driver.find_element_by_xpath('//div')

In [ ]:
len(driver.find_elements_by_xpath('//ol//li//li/label')) # 이 안에 메일이 들어가있음.

In [ ]:
for _ in driver.find_elements_by_xpath('//ol//li//li/label'):
    print(_.text)

In [ ]:
# 로그아웃 구현
# 토글버튼 눌러야 로그아웃 버튼이 보임
driver.find_element_by_class_name('gnb_my').click()

In [ ]:
# 클릭하려고 하니 오류 발생 - interactable 하지 않다!
driver.find_element_by_xpath('//a[@class="gnb_btn_login"]').click()

In [ ]:
# / -> 자식, // -> 자손, [@파라미터='값'] : xpath 양식.
# /div -> (dom의 최상단) root 노드의 자식 div

In [ ]:
logout = driver.find_element_by_xpath('//a[@class="gnb_btn_login"]')

In [ ]:
logout.is_displayed(), logout.is_enabled(), logout.is_selected()

In [ ]:
# is_displayed가 True, is_enabled가 True이야지만 작동할 수 있음?
# 이걸 기다리기 위한 2가지 방법
# 1) time(sleep)
# 2) explict wait, implict wait - 이렇게 2가지 방법이 있다.

In [ ]:
driver.close()

In [ ]:
driver = Chrome()